# Project #2: Investigate a Dataset (Soccer Database)

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
# Introduction

>### About the dataset
- I've selected "Soccer Database" which contains huge real data about European leagues like matches, teams, countries,<br> players, and even the attributes of each team, and each player also, in a period of seasons from _2008 to 2016_, as these<br> data are stored in SQL database, what lets me experience more advanced data connections and enhance my analysis<br> skills, as it consists of **7** tables and **222,796‬** total records (rows).

>### Questions:

1. Which are the top 10 teams in top famous European leagues?
2. Which teams won the title the most?
3. What are attributes lead to the most victorious?
4. Which is the most competitive league in Europe?
5. Which teams always rank in last 3 places in European leagues?
6. What attributes the low-rank teams are missing to improve their performance?
7. Who are the top 10 players in Europe for seasons (2008~2016)?
8. What are attributes lead to be a top player?


In [ ]:
# Use this cell to set up import statements for all of the packages that you
# plan to use.

# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html


import pandas as pd
import numpy as np
import sqlite3 as sql
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


<a id='wrangling'></a>
# Data Wrangling

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
# types and look for instances of missing or possibly errant data.

try:
    conn = sql.connect('soccer_database.sqlite')

    print("Database connected successfully", "\U0001F44D")

except sqlite3.Error as error:
    
    print("Database connection failed!", "\U0001F44E")


____________________________________
### **Checking database tables names:**
(**To decide which tables I will use for my analysis**) 
____________________________________

In [ ]:

tables = pd.read_sql_query("""
                            SELECT name
                            FROM sqlite_master
                            WHERE type='table'
                            """, conn)
tables

____________________________________
### Assessing **"Team"** table
____________________________________

In [ ]:
team = pd.read_sql_query("SELECT * FROM Team", conn)

print('\nColumn names of\'Team\' table are:\n')
for col in team.columns:
    print(col)

print('\n'+'*'*50)
print('Number of \'Team\' table columns is: ', (team.shape[1]), '\nNumber of \'Team\' table rows is: ', (team.shape[0]))
print('*'*50)

print('\nNumber of NULL values in \'Team\' table are:')
print(team.isnull().sum())
print('\n'+'*'*50)


print('Number of duplicated data in \'Team\' table is:', team.duplicated().sum())
print('*'*50,'\n')

print('Chunk of \'Team\' table data (first 5 records):')

team.head()

In [ ]:
#Checking table attributes individually for abstracting connection between tables and the schema

print('\nColumn names of\'Team\' table are:\n')
for col in team.columns:
    print(col)

Ok! here I dicovered that number of records in "Team" table is:  299 record

Let's give it more deepe check...

In [ ]:
#checking number of unique name records
print(len(team['team_long_name'].unique().tolist()))

#checking number of unique IDs records
print(len(team['team_api_id'].unique().tolist()))

Oops! Here I discovered that the number of (unique) team name is only **296**,

while their IDs are **299**, and there's no NULLS in "team_long_name" (As per our check above).

_Conclusion:_

It means that there's a "team_long_name" recorded twice, and got TWO different IDs, 

So I have to delete one of theses records to be only one unique ID for that team.

In [ ]:
#Drop the whole row if duplicated record exists in "team_long_name" column
team = team.drop_duplicates(subset='team_long_name')

#Checking new update
print(len(team['team_long_name'].unique().tolist()))
print(len(team['team_api_id'].unique().tolist()))

Here we go!
So my conclusion was right,and now it's clear! _Welldone!_

____________________________________
### Assessing **"Team_Attributes"** table
____________________________________

In [ ]:
team_atts = pd.read_sql_query("SELECT * FROM Team_Attributes", conn)

print('\n'+'*'*50)
print('Number of \'Team_Attributes\' table columns is: ', (team_atts.shape[1]), '\nNumber of \'Team_Attributes\' table rows is: ', (team_atts.shape[0]))
print('*'*50)

print('\nNumber of NULL values in \'Team_Attributes\' table are:')
print(team_atts.isnull().sum())
print('\n'+'*'*50)


print('Number of duplicated data in \'Team_Attributes\' table is:', team_atts.duplicated().sum())
print('*'*50,'\n')

print('Chunk of \'Team_Attributes\' table data (first 5 records):')

team_atts.head()

In [ ]:
# Checking table attributes individually for abstracting connection between tables and the schema

print('\nColumn names of\'Team_Attributes\' table are:\n')
for col in team_atts.columns:
    print(col)

____________________________________
### Assessing **"League"** table
____________________________________

In [ ]:
league = pd.read_sql_query("SELECT * FROM League", conn)

print('\nColumn names of\'League\' table are:\n')
for col in league.columns:
    print(col)
    
print('\n'+'*'*50)
print('Number of \'League\' table columns is: ', (league.shape[1]), '\nNumber of \'League\' table rows is: ', (league.shape[0]))
print('*'*50)

print('\nNumber of NULL values in \'League\' table is:')
print(league.isnull().sum())
print('\n'+'*'*50)

print('Number of duplicated data in \'League\' table is:', league.duplicated().sum())

print('*'*50,'\n')

# Just League (Not head) as in shape I could notice after the run that it's only 11 rows ;)
league

In [ ]:
# Checking table attributes individually for abstracting connection between tables and the schema

print('\nColumn names of\'League\' table are:\n')
for col in league.columns:
    print(col)

____________________________________
### Assessing **"Country"** table
____________________________________

In [ ]:
country = pd.read_sql_query("SELECT * FROM Country", conn)

print('\nColumn names of\'Country\' table are:\n')
for col in country.columns:
    print(col)

print('\n'+'*'*50)
print('Number of \'Country\' table columns is: ', (country.shape[1]), '\nNumber of \'Country\' table rows is: ', (country.shape[0]))
print('*'*50)

print('\nNumber of NULL values in \'Country\' table are:')
print(country.isnull().sum())
print('\n'+'*'*50)


print('Number of duplicated data in \'Country\' table is:', country.duplicated().sum())
print('*'*50,'\n')

country

In [ ]:
# Checking table attributes individually for abstracting connection between tables and the schema

print('\nColumn names of\'Country\' table are:\n')
for col in country.columns:
    print(col)

____________________________________
### Assessing **"Match"** table
____________________________________

In [ ]:
match = pd.read_sql_query("SELECT * FROM Match", conn)

print('\n'+'*'*50)
print('Number of \'Match\' table columns is: ', (match.shape[1]), '\nNumber of \'Country\' table rows is: ', (match.shape[0]))
print('*'*50)

print('\nNumber of NULL values in \'Match\' table are:')
print(match.isnull().sum())
print('\n'+'*'*50)


print('Number of duplicated data in \'Match\' table is:', match.duplicated().sum())
print('*'*50,'\n')

print('Chunk of \'Match\' table data (first 5 records):')
match.head()

In [ ]:
# Checking table attributes individually for abstracting connection between tables and the schema

print('\nColumn names of\'Match\' table are:\n')
for col in match.columns:
    print(col)

Database Diagram, what shows us the Star-Schema,

and how attributes are connected together through the fact-table (Match)

<img src="soccer_diagram.png">

____________________________________
### Assessing **"Player"** table
____________________________________

In [ ]:
player = pd.read_sql_query("SELECT * FROM Player", conn)

print('\nColumn names of\'Player\' table are:\n')
for col in player.columns:
    print(col)

print('\n'+'*'*50)
print('Number of \'Player\' table columns is: ', (player.shape[1]), '\nNumber of \'Player\' table rows is: ', (player.shape[0]))
print('*'*50)

print('\nNumber of NULL values in \'Player\' table are:')
print(player.isnull().sum())
print('\n'+'*'*50)


print('Number of duplicated data in \'Player\' table is:', player.duplicated().sum())
print('*'*50,'\n')

print('Chunk of \'Player\' table data (first 5 records):')

player.head()

____________________________________
### Assessing **"Player_Attributes"** table
____________________________________

In [ ]:
player_atts = pd.read_sql_query("SELECT * FROM Player_Attributes", conn)

print('\n'+'*'*50)
print('Number of \'Player_Attributes\' table columns is: ', (player_atts.shape[1]), '\nNumber of \'Player_Attributes\' table rows is: ', (player_atts.shape[0]))
print('*'*50)

print('\nNumber of NULL values in \'Player_Attributes\' table are:')
print(player_atts.isnull().sum())
print('\n'+'*'*50)


print('Number of duplicated data in \'Player_Attributes\' table is:', player_atts.duplicated().sum())
print('*'*50,'\n')

print('Chunk of \'Player_Attributes\' table data (first 5 records):')

player_atts.head()



# Data Cleaning & Modification

Below I sliced the data from multiple tables and **Join** them together

(It's more like when you drop the unwanted columns of databases from flat files)

So here I will get all matches with** ***League names*** and ***Team names***

In [ ]:
match_with_team_name = pd.read_sql_query("""
                                            SELECT
                                            Match.id,
                                            League.name AS League_Name,
                                            season AS Season,
                                            home_team_api_id As Home_ID,
                                            home.team_long_name As Home_Team,
                                            home_team_goal,
                                            away_team_goal,
                                            away.team_long_name AS Away_Team,
                                            away_team_api_id As Away_ID
                                            FROM Match
                                            JOIN League on League.id = Match.league_id
                                            JOIN Team AS team_id on team_id.team_api_id = Match.home_team_api_id
                                            JOIN Team As home on home.team_api_id = Match.home_team_api_id
                                            JOIN Team As away on away.team_api_id = Match.away_team_api_id
                                                                                                                                    
                                            """, conn)
                                           

In [ ]:
# Checking dataframe
match_with_team_name.head()

### **Here we go!**
My plan now is to add **points** columns _(__home_team_points__)_ & _(__away_team_points__)_.
What will be calculated as follwing:

Team who scores more goals than the other one, gets ***3*** points and the other one gets ***0*** point.
If two teams goals are equal, so both get ***1*** point.

I will start by "Home" attributes first , then "Away" attributes
Let's see how! ;)


## Home Teams

In [ ]:
# Home_Team points

match_with_team_name.loc[match_with_team_name['home_team_goal'] > match_with_team_name['away_team_goal'] , 'home_team_points'] = 3
match_with_team_name.loc[match_with_team_name['home_team_goal'] == match_with_team_name['away_team_goal'], 'home_team_points'] = 1
match_with_team_name.loc[match_with_team_name['home_team_goal'] < match_with_team_name['away_team_goal'], 'home_team_points'] = 0

In [ ]:
# Check after adding column what calculate Home_Team points

match_with_team_name.head()

### **Perfect!**
Now let's creating more specific dataframe that contains Home_Teams data only

In [ ]:
home_team_data = match_with_team_name[['League_Name','Season','Home_Team','home_team_goal','away_team_goal','home_team_points']]

In [ ]:
#Check...
home_team_data.head()

### **Perfect!**
Now we will add new columns which are **"W"** for (Win), **"D"** for (Draw), and **L** for (Lose).
And they will just like (True or False) attributes.

It will be calculated as the following:

* if home_points = 3, so "W" will have the value of  "1"

* if home_points = 1, so "D" will have the value of  "1"

* if home_points = 0, so "L" will have the value of  "1"


In [ ]:
home_team_data.loc[home_team_data['home_team_points'] == 3, 'W'] = 1
# Filling the Nan values by 0
home_team_data['W'] = home_team_data['W'].fillna(0)
# Converting its data-type to int , as we will use these columns in some calculations
home_team_data['W'] = home_team_data['W'].astype(int)

home_team_data.loc[home_team_data['home_team_points'] == 1, 'D'] = 1
home_team_data['D'] = home_team_data['D'].fillna(0)
home_team_data['D'] = home_team_data['D'].astype(int)

home_team_data.loc[home_team_data['home_team_points'] == 0, 'L'] = 1
home_team_data['L'] = home_team_data['L'].fillna(0)
home_team_data['L'] = home_team_data['L'].astype(int)

# Calculating the goal difference between goals scored and goals recieved
home_team_data['GD'] = home_team_data['home_team_goal'] - home_team_data['away_team_goal']
home_team_data['GD'] = home_team_data['GD'].astype(int)

# Calculating number of "Match Played"
home_team_data['MP'] = (home_team_data['W'] + home_team_data['D'] + home_team_data['L'])

# Also make sure that "points" column is "int" too
home_team_data['home_team_points'] = home_team_data['home_team_points'].astype(int)

In [ ]:
home_team_data.head()

### **Superb!**
Now I will _Groupby_ ***"Season"***, ***"Home_Team"***, and ***"League_Name"***, and _SUM_ of the other columns.

Mmmmm.. So what's the purpose of this step?

Purpose is to make aggregate meaningful data for each Team,
As every (Team) will be recorded ONLY ONCE in every season

matching its records in this season of goals, points and status.

***Let's how it will be done!***

In [ ]:
home_t = home_team_data.groupby(['Season', 'Home_Team','League_Name'], as_index=False).sum()
home_t.head()

### **Great!**
Let's try to filter it and check the output

In [ ]:
filter_test = home_t[home_t['Home_Team']=='Arsenal']
filter_test.head()

In [ ]:
# Now I will just rename some columns, just to be perfect invidual table, and for any merge or concat sake, later ;)
# As I'm in home teams info, so "away_team_goal" here will act as "Goals Against", so I will rename it to "GA"

home_t= home_t.rename(columns={'Home_Team': 'Team_Name', 'home_team_goal': 'goals','away_team_goal': 'GA', 'home_team_points': 'points'})
home_t.head()

### Good!
Let's now do exact the same to Away attributes ;)

## Away Teams

In [ ]:
# Away_Team points

match_with_team_name.loc[match_with_team_name['away_team_goal'] > match_with_team_name['home_team_goal'] , 'away_team_points'] = 3
match_with_team_name.loc[match_with_team_name['away_team_goal'] == match_with_team_name['home_team_goal'], 'away_team_points'] = 1
match_with_team_name.loc[match_with_team_name['away_team_goal'] < match_with_team_name['home_team_goal'], 'away_team_points'] = 0

In [ ]:
away_team_data = match_with_team_name[['League_Name','Season','Away_Team','away_team_goal','home_team_goal','away_team_points']]

In [ ]:
away_team_data.loc[away_team_data['away_team_points'] == 3, 'W'] = 1
away_team_data['W'] = away_team_data['W'].fillna(0)
away_team_data['W'] = away_team_data['W'].astype(int)

away_team_data.loc[away_team_data['away_team_points'] == 1, 'D'] = 1
away_team_data['D'] = away_team_data['D'].fillna(0)
away_team_data['D'] = away_team_data['D'].astype(int)

away_team_data.loc[away_team_data['away_team_points'] == 0, 'L'] = 1
away_team_data['L'] = away_team_data['L'].fillna(0)
away_team_data['L'] = away_team_data['L'].astype(int)


away_team_data['GD'] = away_team_data['away_team_goal'] - away_team_data['home_team_goal']
away_team_data['GD'] = away_team_data['GD'].astype(int)


away_team_data['MP'] = (away_team_data['W'] + away_team_data['D'] + away_team_data['L'])


away_team_data['away_team_points'] = away_team_data['away_team_points'].astype(int)

In [ ]:
away_t = away_team_data.groupby(["Season", "Away_Team", "League_Name"], as_index=False).sum()

In [ ]:
away_t= away_t.rename(columns={'Away_Team': 'Team_Name', 'away_team_goal': 'goals','home_team_goal':'GA', 'away_team_points': 'points'})
away_t.head()

In [ ]:
print(away_t.isnull().sum())
print('\n'+'*'*50)

In [ ]:
filter_test_2 = away_t[away_t['Team_Name']=='Arsenal']
filter_test_2.head()

## League Standings Dataframe

### Now I have two completed tables (Home & Away) for each Team in each Season

So, I need to union the dataframes, but to avoid any logic problems, as the tables are a mix of categorical & dependant numerical records, So I decided to slice the tables to categorical first, then append the metrics attributes

Let's see how I will figure it out

In [ ]:
home_categorical_slice = home_t[['League_Name', 'Season', 'Team_Name']]
home_categorical_slice.head()

In [ ]:
away_categorical_slice = away_t[['League_Name', 'Season', 'Team_Name']]
away_categorical_slice.head()

### Well!
Now I'll ***concat*** the two categorical tables

Note: "Concat" appends the rows of the _right dataframe_ to the _left_ one

Let's check the shape of each table first


In [ ]:
print(home_categorical_slice.shape)
print(away_categorical_slice.shape)

So, _Away_ table rows will come below the _Home_ rows

So the shape will be the sum of rows, and same col. number (1478 + 1478 = 2956)

Let's see

In [ ]:
leagues_standings = pd.concat([home_categorical_slice, away_categorical_slice], ignore_index=True)
leagues_standings.head()

In [ ]:
leagues_standings.shape

In [ ]:
print(leagues_standings.isnull().sum())
print('\n'+'*'*50)

### Here we go!
I will add the rest of the columns by Summing both records of _Home_ & _Away_

In [ ]:
leagues_standings['MP'] = (home_t['MP'] + away_t['MP'])


leagues_standings['W'] = (home_t['W'] + away_t['W'])

leagues_standings['D'] = (home_t['D'] + away_t['D'])

leagues_standings['L'] = (home_t['L'] + away_t['L'])


leagues_standings['Goals'] = (home_t['goals'] + away_t['goals'])

leagues_standings['GA'] = (home_t['GA'] + away_t['GA'])

leagues_standings['GD'] = (home_t['GD'] + away_t['GD'])


leagues_standings['Points'] = (home_t['points'] + away_t['points'])


In [ ]:
leagues_standings.head()

In [ ]:
# Checking Nulls in our dataframe to make it clean, if needed

print(leagues_standings.isnull().sum())
print('\n'+'*'*50)

In [ ]:
# Cleaning the data by dropping all rows with Nan values
leagues_standings.dropna(subset = ['W'], inplace=True)

print(leagues_standings.isnull().sum())
print('\n'+'*'*50)

In [ ]:
# And convert datatype to int to look more reasonable

leagues_standings['MP'] = leagues_standings['MP'].astype(int)
leagues_standings['W'] = leagues_standings['W'].astype(int)
leagues_standings['D'] = leagues_standings['D'].astype(int)
leagues_standings['L'] = leagues_standings['L'].astype(int)
leagues_standings['GA'] = leagues_standings['GA'].astype(int)
leagues_standings['GD'] = leagues_standings['GD'].astype(int)
leagues_standings['Goals'] = leagues_standings['Goals'].astype(int)
leagues_standings['Points'] = leagues_standings['Points'].astype(int)


### Perfecto!
Here I decided to create a function that returns any standings for any league in any season (2008 ~ 2015)

It's perfect for retrieving any league standings, and also for further uses

In [ ]:
def League_stands(league_name,season):

    df = leagues_standings[(leagues_standings['League_Name']==f'{league_name}') &
    (leagues_standings['Season']==f'{season}')].sort_values(by='Points', ascending=False)

    #Adding rank column, and sort data according to it
    df['Rank'] = df['Points'].rank(method='first', ascending=False).astype(int)
        
    return df

Let's try it!

In [ ]:
League_stands('Spain LIGA BBVA','2008/2009').head()

Seems great! but let me compare it with the real standings on "Google".

In [ ]:
%%html
<img src="laliga-2009.PNG" style="width:835px;height:606px"/>

Wonderful!! Exact the same!!

<a id='eda'></a>
# Exploratory Data Analysis

Now, I decided to set a plan for getting the top 1 team in each league
<br>
This the plan:

1. I chose the top 5 famous leagues , to do my analysis upon them, which are (***Premier League,  Serie A,  La Liga,  Bundesliga,  and Ligue 1***).

2. Pick most team won the league championship in its country, in seasons (2008 ~ 2016).
3. Check the characteristics of top teams in the world.

And this is how I will proceed the plan:

1. Create lists of leagues, Seasons, and Teams to use them in upcoming loops.
2. Create a table that contains the top five teams of each league in every season.
3. Get the most winning team in each league.
4. Gather them together in the data frame.
5. Join these teams to their characteristics in the _Attribute_ table.
6. get the most common average of these characteristics.

____________________________________
### Top 10 Teams over the leagues
____________________________________

First let's get the first top 10 of the five leagues overall, according to their total points


In [ ]:
# Creating the lists
league_list = league['name'].unique().tolist()
season_list = match['season'].unique().tolist()
team_list  = team['team_long_name'].unique().tolist()

In [ ]:
# Creating list of desired leagues only.
top_five_leagues = ['France Ligue 1', 'England Premier League', 'Germany 1. Bundesliga', 'Italy Serie A', 'Spain LIGA BBVA']

This loop below, uses the "_League_stands_" function to fill the desired table

In [ ]:
# This will be like a temp table
df = pd.DataFrame

# This will be our usable table
df_2 = pd.DataFrame(columns=['League_Name','Season','Team_Name','MP','W','D','L','Goals','GA','GD','Points','Rank'])

y=0
while y < (len(league_list)):
    x=0

    while x < (len(season_list)):
       df = League_stands(league_list[y],season_list[x]).head()
       df_2 = df_2.append(df)       
       x+=1

    y+=1

df_2.head()

In [ ]:
# Dropping unneeded columns, to make data more clear
top_10_t = df_2.drop(['MP','W','D','L','Goals','GA','GD','Season','Rank'], axis=1)
top_10_t = top_10_t[top_10_t.League_Name.isin(top_five_leagues)]
top_10_t.head(10)

In [ ]:
top_10_t = top_10_t.groupby(['League_Name','Team_Name'], as_index=False).sum()

# Removing duplicates if exists
top_10_t = top_10_t.drop_duplicates()

# Sorting by most team got more points
top_10_t = top_10_t.sort_values(by='Points', ascending=False, ignore_index=True)
top_10_t = top_10_t.drop(['League_Name','Points'], axis=1)

top_10_t.head(10)

So here I can answer the question:<br>
***Which are the top 10 teams in top famous European leagues?***<br>
and the answer will be:<br>

_1. FC Barcelona<br>
2.	Real Madrid CF<br>
3.	FC Bayern Munich<br>
4.	Arsenal<br>
5.	Manchester United<br>
6.	Paris Saint-Germain<br>
7.	Olympique Lyonnais<br>
8.	Manchester City<br>
9.	Juventus<br>
10.	Atlético Madrid_

____________________________________
### Top-5 Teams (Top 1 of each league in the 5 leagues)
____________________________________

In [ ]:
# This will be like a temp table
df = pd.DataFrame

# This will be our usable table
df_2 = pd.DataFrame(columns=['League_Name','Season','Team_Name','MP','W','D','L','Goals','GA','GD','Points','Rank'])

y=0
while y < (len(league_list)):
    x=0

    while x < (len(season_list)):
       df = League_stands(league_list[y],season_list[x]).head(5)
       df_2 = df_2.append(df)       
       x+=1

    y+=1

df_2

In [ ]:
# Creating list of desired leagues only.
top_five_leagues = ['France Ligue 1', 'England Premier League', 'Germany 1. Bundesliga', 'Italy Serie A', 'Spain LIGA BBVA']

In [ ]:
# Dropping unneeded columns, to make data more clear
top_5_t = df_2.drop(['MP','W','D','L','Goals','GA','GD'], axis=1)
top_5_t = top_5_t[top_5_t.League_Name.isin(top_five_leagues)]
top_5_t

In [ ]:
# Filtering to get only the winners of leagues in every single season (top#1)
num1_t = (top_5_t.loc[top_5_t['Rank'] == 1])
num1_t = num1_t.groupby(['Team_Name', 'League_Name', 'Season'], as_index=False).sum()

num1_t.head()

Below is a little surgery  to count how many times each team was number one,<br>
so, we can get the most league champ

In [ ]:
num1_t['Freq'] = num1_t.groupby('Team_Name')['Team_Name'].transform('count')
#"transform" function above, returns the dataframe with transformed values
#  after applying the function specified in its parameter

num1_t.head()

In [ ]:
# Dropping "Season" & "Points" for better visible table.
num1_t = num1_t.drop(['Season'], axis=1)

num1_t.head()

In [ ]:
# Removing duplicates if exists
num1_t = num1_t.drop_duplicates()

# Sorting by most frequency (team who most was)
num1_t = num1_t.sort_values(by='Freq', ascending=False)

# After sorting, so we need only one team from each league , what will be the most champ winner.
# So any repeated League_Name , will be deleted but keep the first appearance, which is the highest Freq. ;)
num1_t = num1_t.drop_duplicates(subset='League_Name', keep='first', ignore_index=True)

num1_t

So here I can answer the question:<br>
***Which teams won the title the most?***<br>
and the answer will be:<br>
_1. FC Barcelona_<br>
_2. Juventus_<br>
_3. FC Bayern Munich_<br>
_4. Paris Saint-Germain_<br>	
_5. Manchester United_<br>


____________________________________
### Last-5 teams
____________________________________

Doing the same as I did for "Top-5" teams

In [ ]:
# This will be like temp table
df = pd.DataFrame
# This will be our usable table
df_2 = pd.DataFrame(columns=['League_Name','Season','Team_Name','MP','W','D','L','Goals','GA','GD','Points','Rank'])

y=0
while y < (len(league_list)):
    x=0

    while x < (len(season_list)):
       df = League_stands(league_list[y],season_list[x]).tail(5)
       df_2 = df_2.append(df)       
       x+=1

    y+=1

In [ ]:
# Droppig unneeded columns, to make data more clear
last_5_t = df_2.drop(['MP','W','D','L','Goals','GA','GD'], axis=1)
last_5_t = last_5_t[last_5_t.League_Name.isin(top_five_leagues)]

In [ ]:

num20_t = (last_5_t.loc[last_5_t['Rank'] == 18])
# I've made the rank 18 not 20, as I discovered that Bundesliga has no rank 20, so it's exceptional for our case.

num20_t = num20_t.groupby(['Team_Name', 'League_Name', 'Season'], as_index=False).sum()

In [ ]:
num20_t['Freq'] = num20_t.groupby('Team_Name')['Team_Name'].transform('count')

In [ ]:
# Dropping "Season" & "Points" for better visible table
num20_t = num20_t.drop(['Season'], axis=1)

num20_t.head()

In [ ]:
# Removing duplicates
num20_t = num20_t.drop_duplicates()

# Sorting by most frequency (team who most was in last place)
num20_t = num20_t.sort_values(by='Freq', ascending=False)

# After sorting, so we need only one team from each league , what will be the most one ranked last.
# So any repeated League_Name , will be deleted but keep the first appearance, which is the highest Freq. ;)
num20_t = num20_t.drop_duplicates(subset='League_Name', keep='first', ignore_index=True)

num20_t

So here I can answer the question:<br>
***Which teams always rank in last 3 places in European leagues?***<br>
and the answer will be:<br>
_1. Newcastle United_<br>
_2. RC Deportivo de La Coruña_<br>
_3. 1.FC Kaiserslautern_<br>
_4. Lecce_<br>	
_5. SM Caen_<br>

Now it's time for creating some visualizations, to get some observations and conclusions.

I'll plot a bar-chart for The most title-winning teams in each league, in seasons (2008 ~ 2016)

In [ ]:
#Let's try using the "Seaborn" library
# as it supports a much easier way for _legend_ (hue), and I need it here without complexity

colors = ['purple','green','#D83B01', 'blue', 'red']

sns.set_theme(style="whitegrid")
fig, ax = plt.subplots(figsize=(14, 8))
sns.barplot(data=num1_t, hue='League_Name', x='Team_Name', y='Freq', palette=colors, ax=ax).set(title='The most title-winning teams in each league\n-in seasons (2008~2016)')

plt.xlabel('Team name')
plt.ylabel('The number of tournaments won')

for tick in ax.get_xticklabels():
    tick.set_rotation(10)
    
plt.legend(title='League', fontsize='9', title_fontsize='10')
plt.show()



**Observation:**

* _Man.United_ is the most title winner and only won 3 seasons in 7-years.

**Conclusion:**

* _Premier League_ is a very competitive league.



I'll plot a bar-chart for The most teams ranked last in each league, in seasons (2008 ~ 2016)

In [ ]:
colors = ['red', 'purple', '#D83B01', 'green', 'blue']

sns.set_theme(style="whitegrid")
fig, ax = plt.subplots(figsize=(14, 8))
sns.barplot(data=num20_t, hue='League_Name', x='Team_Name', y='Freq', palette=colors, ax=ax).set(title='The teams that achieved the most last places in each league\n-in seasons (2008~2016)')
plt.xlabel('Team name')
plt.ylabel('Number of times ranked in the last places')

for tick in ax.get_xticklabels():
    tick.set_rotation(10)

plt.legend(title='League', fontsize='9', title_fontsize='10')
plt.show()

Now, I will compare the difference of total points between the top-5 and last-5 teams in each league<br>
so I can indicate which is the most competitive league.

In [ ]:
# This will be like temp table
df = pd.DataFrame
# This will be our usable table
df_2 = pd.DataFrame(columns=['League_Name','Season','Team_Name','MP','W','D','L','Goals','GA','GD','Points','Rank'])

y=0
while y < (len(league_list)):
    x=0

    while x < (len(season_list)):
       df = League_stands(league_list[y],season_list[x])
       df_2 = df_2.append(df)       
       x+=1

    y+=1

In [ ]:
df_2 = df_2.groupby(['Team_Name'], as_index=False).sum()
df_2 = df_2.drop(['Season','MP','W','D','L','Goals','GA','GD','Rank'], axis=1)
df_2 = df_2.sort_values(by='Points',ascending = False)

In [ ]:
top5_pts = pd.merge(num1_t,df_2, how='inner', left_on = 'Team_Name',right_on = 'Team_Name')
top5_pts = top5_pts.drop(['Points_x','Rank','Freq','League_Name_y'], axis=1)

last5_pts = pd.merge(num20_t,df_2, how='inner', left_on = 'Team_Name',right_on = 'Team_Name')
last5_pts = last5_pts.drop(['Points_x','Rank','Freq','League_Name_y'], axis=1)

In [ ]:
top_dwn_diff = pd.merge(top5_pts,last5_pts, how='inner', left_on = 'League_Name_x',right_on = 'League_Name_x')
top_dwn_diff = top_dwn_diff.drop(['Team_Name_x','Team_Name_y'], axis=1)
top_dwn_diff = top_dwn_diff.rename(columns={'League_Name_x': 'League_Name', 'Points_y_x': 'Top 5 points', 'Points_y_y': 'Last 5 points'})
top_dwn_diff['Pts. Diff.'] = top_dwn_diff['Top 5 points'] - top_dwn_diff['Last 5 points']

top_dwn_diff

In [ ]:
#Let's use the "Matplotlib" library, as it's much flexible in customizations

#Setting labels locations
x = np.arange(len((top_dwn_diff['League_Name'].unique().tolist())))

#Creating variable called "width" to use it for the width of the bars.
width = 0.4  

fig, ax = plt.subplots()
fig.set_size_inches(10.5, 7.5)

# Top 5 bars.
ax.bar(x - width/2, top_dwn_diff['Top 5 points'], width, label='Top rank team')

# Last 5 bars.
ax.bar(x + width/2, top_dwn_diff['Last 5 points'], width, label='Last rank team')


# Adding some labels, titles and x-axis tick labels.
ax.set_title('Difference of total points, between Top & Last teams in each league\n-in seasons (2008~2016)')
plt.xlabel('League Name')
ax.set_ylabel('Points')
ax.set_xticks(x)
ax.set_xticklabels(top_dwn_diff['League_Name'])


for tick in ax.get_xticklabels():
    tick.set_rotation(10)

ax.legend()

fig.tight_layout()

plt.show()

**Observation:**

* _Premier League_ difference point is almost the half.

Let's make it more observant! ;)

In [ ]:
#This function is to add nottation above each bar, by it's value
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')


# Creating data list on which bar chart will be plot
x = top_dwn_diff['League_Name'].tolist()
y = top_dwn_diff['Pts. Diff.'].tolist()

colors = ['purple', 'green', '#D83B01', 'blue','red']

# Setting figure size by using figure() function 
plt.figure(figsize = (15, 7))
plt.bar(x, y, color = colors)
    
# Calling the function to add the labels above each bar
addlabels(x, y)
 
plt.title("Difference of total points, between Top & Last teams in each league\n-in seasons (2008~2016)")
plt.xlabel("League name")
plt.ylabel("Points")
      
plt.show()

**Observation:**

* _Premier League_ has the lowest difference of point between top and last teams.<br>

**Conclusion:**

* _Premier League_ is the most competitive league in Europe.

So here I can answer the question:<br>
***Which is the most competitive league in Europe?***<br>
and the answer will be:<br>
_England Premier League_

### Good!
Now I will join "_num1_t_" (what contains top-5 teams), with "$team$" table to get teams IDs,<br>
so I can join the "_team_atts_", and get the metrics of the top teams.

In [ ]:
top5_atts = pd.merge(left=num1_t, right=team, how='left', left_on='Team_Name', right_on='team_long_name')

# Drop unwanted columns.
top5_atts = top5_atts.drop(['League_Name','Rank','Freq','id','team_fifa_api_id','team_long_name','team_short_name'], axis=1)

top5_atts

In [ ]:
#Joining my table with "Team_Attributes" table
top5_atts = top5_atts.merge(team_atts, how='left', left_on='team_api_id', right_on='team_api_id')

In [ ]:
#Dropping unusful cols.
top5_atts = top5_atts.drop(['id','team_fifa_api_id','buildUpPlaySpeedClass','buildUpPlayDribblingClass',
                                'chanceCreationShootingClass','chanceCreationPositioningClass','defenceAggressionClass',
                                'defenceTeamWidthClass','defenceDefenderLineClass','buildUpPlayPassingClass',
                                'buildUpPlayPositioningClass','chanceCreationPassingClass','chanceCreationCrossingClass',
                                'defencePressureClass'], axis=1)

In [ ]:
# Getting the average of each attribute of the top teams
top5_atts.fillna(value=top5_atts['buildUpPlayDribbling'].mean(), inplace=True)

print(top5_atts.isnull().sum())

In [ ]:
#Groupping by Team_Name and ID
top5_atts_avg = top5_atts.groupby(['Team_Name','team_api_id'],as_index=False).mean()
top5_atts_avg.head()

In [ ]:
#Selecting only metrics cols. (dropping first 2, and start from 3rd to the end of cols.)
#Remember that Python does not slice inclusive of the ending index.
top5_means = (top5_atts_avg.iloc[:,3:]).mean() 
top5_means.head()

### Now I just got the attributes of "Top Teams"
### Let's do the same for the "Down Teams"!

In [ ]:
last5_atts = pd.merge(left=num20_t, right=team, how='left', left_on='Team_Name', right_on='team_long_name')

# Drop unwanted columns.
last5_atts = last5_atts.drop(['League_Name','Rank','Freq','id','team_fifa_api_id','team_long_name','team_short_name'], axis=1)

last5_atts

In [ ]:
last5_atts = last5_atts.merge(team_atts, how='left', left_on='team_api_id', right_on='team_api_id')

In [ ]:
last5_atts = last5_atts.drop(['id','team_fifa_api_id','buildUpPlaySpeedClass','buildUpPlayDribblingClass',
                                'chanceCreationShootingClass','chanceCreationPositioningClass','defenceAggressionClass',
                                'defenceTeamWidthClass','defenceDefenderLineClass','buildUpPlayPassingClass',
                                'buildUpPlayPositioningClass','chanceCreationPassingClass','chanceCreationCrossingClass',
                                'defencePressureClass'], axis=1)

In [ ]:
print(last5_atts.isnull().sum())

In [ ]:
last5_atts.fillna(value=last5_atts['buildUpPlayDribbling'].mean(), inplace=True)

print(last5_atts.isnull().sum())

In [ ]:
last5_atts_avg = last5_atts.groupby(['Team_Name','team_api_id'],as_index=False).mean()
last5_atts_avg.head()

In [ ]:
last5_atts_means = (last5_atts_avg.iloc[:,3:]).mean()

last5_atts_means

### Here we go!
I've chosen the Radar Chart as I believe it's the best representative for attribute distribution And can compare through it difference of top & down teams abilities.

In [ ]:
# Setting lables to be the angles of the radar
labels = [
    'buildUpPlaySpeed',
    'buildUpPlayDribbling',
    'buildUpPlayPassing',
    'chanceCreationPassing',
    'chanceCreationCrossing',
    'chanceCreationShooting',
    'defencePressure',
    'defenceAggression',
    'defenceTeamWidth']

# Creating the chart
top5_stat = top5_means.values

angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="polar")

# Theta found with five different angles, but first is duplicated , so:
theta = np.arange(len(top5_stat) + 1) / float(len(top5_stat)) * 2 * np.pi

values = top5_means.values
values = np.append(values, values[0])


# Drawing the polygon, and marking the points for each angle/value combination.
ax.plot(theta, values, color="darkblue", marker="o", label="top teams stats")
plt.xticks(theta[:9], labels, color='black', size=12)

# Increasing the distance of the labels on the plot
ax.tick_params(pad=10) 

# Fill the area of the polygon by blue color and giving it some transparency
ax.fill(theta, values, 'blue', alpha=0.5)
ax.grid(True)
 
plt.title('Top Teams Skills\n')
plt.show()

**Observation:**

* Top teams are much skilled in _Creating shooting chances_, _Defence Pressure_, _Defense Agression_ and _Defence Width_ has the lowest difference of point between top and last teams.<br>

**Concluseion:**

I could answer the question:<br>
***What are attributes lead to the most victorious?***<br>
and the answer will be:<br>
To focuse more in 
* _Creating shooting chances_
* _Defence Pressure_
* _Defense Agression_
* _Defence Width_

And now for the doen ranked teams

In [ ]:
# Setting lables to be the angles of the radar
labels = [
    'buildUpPlaySpeed',
    'buildUpPlayDribbling',
    'buildUpPlayPassing',
    'chanceCreationPassing',
    'chanceCreationCrossing',
    'chanceCreationShooting',
    'defencePressure',
    'defenceAggression',
    'defenceTeamWidth']

# Creating the chart
last5_atts = last5_atts_means.values

angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="polar")


angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)

# Theta found with five different angles, but first is duplicated , so:
theta = np.arange(len(last5_atts) + 1) / float(len(last5_atts)) * 2 * np.pi

values = last5_atts_means.values
values = np.append(values, values[0])




# Drawing the polygon, and marking the points for each angle/value combination.
ax.plot(theta, values, color="darkorange", marker="o", label="top teams stats")
plt.xticks(theta[:9], labels, color='black', size=12)

# Increasing the distance of the labels on the plot
ax.tick_params(pad=10) 

# Fill the area of the polygon by blue color and giving it some transparency
ax.fill(theta, values, 'orange', alpha=0.5)
ax.grid(True)

plt.title('Down Teams Skills\n')
plt.show()

In [ ]:
# Creating the chart
top5_stat = top5_means.values

angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="polar")



# Theta found with five different angles, but first is duplicated , so:
theta = np.arange(len(top5_stat) + 1) / float(len(top5_stat)) * 2 * np.pi

values = top5_means.values
values = np.append(values, values[0])




# Drawing the polygon, and marking the points for each angle/value combination.
ax.plot(theta, values, color="darkblue", marker="o", label="Top Teams Skills")
plt.xticks(theta[:9], labels, color='black', size=12)

# Increasing the distance of the labels on the plot
ax.tick_params(pad=10) 

# Fill the area of the polygon by blue color and giving it some transparency
ax.fill(theta, values, 'blue', alpha=0.5)

# Creating the chart
last5_atts = last5_atts_means.values

angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)

angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)

# Theta found with five different angles, but first is duplicated , so:
theta = np.arange(len(last5_atts) + 1) / float(len(last5_atts)) * 2 * np.pi

values = last5_atts_means.values
values = np.append(values, values[0])




# Drawing the polygon, and marking the points for each angle/value combination.
ax.plot(theta, values, color="darkorange", marker="o", label="Down Teams Skills")
plt.xticks(theta[:9], labels, color='black', size=12)

# Increasing the distance of the labels on the plot
ax.tick_params(pad=10) 

# Fill the area of the polygon by blue color and giving it some transparency
ax.fill(theta, values, 'orange', alpha=0.5)

ax.grid(True)
plt.title('Top vs Down teams\n')

# Showing the legend, using the label of the line plot (useful when there is more than 1 polygon)
plt.legend() 
plt.show()


**Observation:**

* Top teams are more skilled in  _Defence Pressure_ and _Defense Agression_ than down ranked teams.<br>

**Conclusion:**

I could answer the question:<br>
***What attributes the low-rank teams are missing to improve their performance?***<br>
and the answer will be:<br>
Generally, they miss focusing on the "Defense" skills, speciallyon 

* _Defence Pressure_
* _Defense Agression_


____________________________________
### Creating **Players** table Joining their skills in the **Attributes** table<br>
(only needed skilly)
____________________________________

In [ ]:
player_atts_names = pd.read_sql_query("""
                                            SELECT
                                            Player.player_name AS Player_Name, 
                                            pa.crossing,
                                            finishing,
                                            heading_accuracy,
                                            dribbling,
                                            free_kick_accuracy,
                                            long_passing,
                                            ball_control,
                                            acceleration,
                                            sprint_speed,
                                            agility,
                                            balance,
                                            shot_power,
                                            jumping,
                                            stamina,
                                            strength,
                                            long_shots,
                                            overall_rating
                                            FROM Player
                                            JOIN Player_Attributes As pa on pa.player_api_id = Player.player_api_id                                      
                                                                                                                             
                                            """, conn)

In [ ]:
player_atts_names.head()

In [ ]:
player_atts_names = player_atts_names.drop_duplicates()
player_atts_names.shape
player_atts_names.head()
player_atts_names = player_atts_names.groupby(['Player_Name'], as_index=False).mean()

In [ ]:
#Sorting by total Overall rating in all 7-seasons
player_atts_names = player_atts_names.sort_values(by='overall_rating', ascending=False)

In [ ]:
player_atts_names.head(10)

In [ ]:
top_10_players_names = (player_atts_names['Player_Name']).head(10)
top_10_players_names

So here I can answer the question:<br>
***Who are the top 10 players in Europe for seasons (2008~2016)?***<br>
and the answer will be:<br>

_1. Lionel Messi<br>
2.	Cristiano Ronaldo<br>
3.	Franck Ribery<br>
4.	Zlatan Ibrahimovic<br>
5.	Xavi Hernandez<br>
6.	Arjen Robben<br>
7.	Andres Iniesta<br>
8.	Iker Casillas<br>
9.	Wayne Rooney<br>
10.	Philipp Lahm_

In [ ]:
top10_plr = player_atts_names.head(10)
top10_plr = (top10_plr.iloc[:,1:17]).mean()

top10_plr

In [ ]:
sorted = top10_plr.sort_values(ascending=False)

In [ ]:
sorted.head()

In [ ]:
top_plr_stats = sorted.values
lbls=[
'agility',
'ball_control',
'acceleration',
'dribbling',
'sprint_speed',
'balance',
'stamina',
'crossing',
'long_shots',
'long_passing',
'shot_power',
'free_kick_accuracy',
'finishing',
'strength',
'jumping',
'heading_accuracy']

sns.set_theme(style="whitegrid")
fig, ax = plt.subplots(figsize=(14, 8))
sns.barplot(x=top_plr_stats, y=lbls, palette = 'rocket').set(title='\nSkills Of Top Players\n')
plt.xlabel('Average')
plt.ylabel('Ability (Skill)')
plt.show()

So here I can answer the question:<br>
***What are attributes lead to be a top player?***<br>
and the answer will be:<br>
To be mostly skilled at the following abilities:<br>
_1. Agility<br>
2.	Ball Control<br>
3.	Acceleration<br>
4.	Dribbling<br>
5.	Sprint Speed<br>_

<a id='conclusions'></a>
# Conclusions

## **Results:**

As per my analysis, I could extract many significant results and info about:<br> 
* Top teams (who won the title the most).
* Down teams (who rank the last place the most).
* The reasons behind their strengths (high defense ability) and weaknesses (low defense ability).
* Detecting the level of competitively of each league (difference of points between top & low, how many times top teams won the title over the 7 years).
* Spotting the most powerful players (who got the highest overall rating), and skills behind their talents (agility, ball control, acceleration…etc.) that may be being missed in the weaker players.

According to my data exploration and analysis for the soccer dataset, I believe that I can now answer all the questions as following:

> **1.	Which are the top 10 teams in top famous European leagues?**<br>

_1. FC Barcelona<br>
2.	Real Madrid CF<br>
3.	FC Bayern Munich<br>
4.	Arsenal<br>
5.	Manchester United<br>
6.	Paris Saint-Germain<br>
7.	Olympique Lyonnais<br>
8.	Manchester City<br>
9.	Juventus<br>
10.	Atlético Madrid_

> **2.	Which teams won the title the most?**<br>

_1.	FC Barcelona<br>
2.	Juventus<br>
3.	FC Bayern Munich<br>
4.	Paris Saint-Germain<br>
5.	Manchester United_

> **3.	What are attributes lead to the most victorious?**<br>

_Most interesting and important attributes are the creating of shooting chances and defense abilities._

> **4.	Which is the most competitive league in Europe?**<br>

_England Premier League._

> **5.	Which teams always rank in last 3 places in European leagues?**<br>

_1. Newcastle United<br>
2. RC Deportivo de La Coruña<br>
3. 1.FC Kaiserslautern<br>
4. Lecce<br>	
5. SM Caen_

> **6.	What attributes the low-rank teams are missing to improve their performance?**<br>

_Actually they miss the most powerful attribute in top teams which is the defense pressure and aggression._

> **7.	Who are the top 10 players in Europe for seasons (2009~2016)?**<br>

_1.	Lionel Messi<br>
2.	Cristiano Ronaldo<br>
3.	Franck Ribery<br>
4.	Zlatan Ibrahimovic<br>
5.	Xavi Hernandez<br>
6.	Arjen Robben<br>
7.	Andres Iniesta<br>
8.	Iker Casillas<br>
9.	Wayne Rooney<br>
10.	Philipp Lahm_<br>

> **8.	What are attributes lead to be a top player?**<br>

_According to my analysis the most attributes to be developed for being one the best players, are: 
•	Agility
•	Ball control
•	Acceleration
•	Dribbling
•	Sprint spee_


## **Limitations:**

During my discovery in investigating soccer dataset, I noticed some limitations as:<br> 
* A little defect in the database schema, as there are no keys matching the “Player” and “Player Attribute” tables with the “Match” table, so they act as two different databases and are not connected in any way.
* There’s no direct key between the “Match” table and “Team Attribute”.
* All tables are connected through foreign keys only on both sides.
* The missing of the home & away players data in the “Match” table, which makes it faded to know the players who participated in the match.